# **Arkouda: Interactive Analytics in Python on Petascale Datasets**

## **Environment Setup**
Make sure all necessary dependencies are imported, and that the Jupyter Notebook is properly connected to an Arkouda server instance as shown below.

If running the server locally, the server name string can be omitted or replaced with "localhost" if omission doesn't work. 

Port number should always be `5555` by default, unless specified otherwise when launching the `arkouda_server` executable.

In [1]:
import arkouda as ak
import arachne as ar

    _         _                   _       
   / \   _ __| | _____  _   _  __| | __ _ 
  / _ \ | '__| |/ / _ \| | | |/ _` |/ _` |
 / ___ \| |  |   < (_) | |_| | (_| | (_| |
/_/   \_\_|  |_|\_\___/ \__,_|\__,_|\__,_|
                                          

Client Version: v2023.11.15


In [2]:
ak.connect("n19", 5555)

connected to arkouda server tcp://*:5555


## **Reading In Data**
Arkouda supports reading data in from various types including:
- HDF5 - hierarchical, "file directory"-like, data storage for large files
- Apache Parquet - column-oriented data storage format for efficient data compression
- CSV - comma-delimited text files

CSVs are usually good for small-scale data whereas distributed filetypes such as HDF5 and Parquet are preferred for large-scale data. Below, we read in an HDF5 dataset and convert it into an Arkouda dataframe.

Our dataset for this example is composed of links between neurons in a fly brain with data specifying the transmitting neuron and the receiving one. The three-dimensional coordinates for the locations of the links and neurons are specified was well as the size of these bounding boxes. The labels for the type of link between neurons is present as well. The confidence value for the label type is present as well.

![title](connectome.png)

In [3]:
connectome_links = ak.read("/scratch/users/oaa9/testing_files/data/connectome/edges*")
connectome_links = ak.DataFrame(connectome_links)

## **Arkouda Dataframes**
Arkouda dataframes are very similar to Pandas ones. In Arkouda, dataframes are basically a wrapper to dictionaries where the name of the column is a key and the value is an array that represents columnar values. In Arkouda, each column is a `pdarray` to facilitate array operations on large array data.

In [4]:
connectome_links

,bounding_box_size_x,bounding_box_size_y,bounding_box_size_z,bounding_box_start_x,bounding_box_start_y,bounding_box_start_z,confidence,dst_id,dst_neuron_id,location_x,location_y,location_z,src_id,src_neuron_id,type_label
0,56,76,31,366516,51943,31,0.829045,5197731597,636356250,366544,51981,256,5197731596,621784456,2
1,77,62,30,437208,205041,30,0.972310,20521871940,5719183727,437246,205072,256,20521871941,5762854802,2
2,72,66,27,420177,181838,27,0.968344,18216894471,4932569891,420213,181871,256,18216894472,4976182692,2
3,80,62,26,366975,74432,26,0.978425,7441911475,1320603529,367015,74463,256,7441911474,1349644059,2
4,80,52,27,451013,132460,27,0.951494,13281560451,3303155953,451053,132486,256,13281560452,3332269800,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149871664,29,31,4,350941,104477,4,0.754649,616801202162,99595655834,350955,104492,5119,616801202163,99595655802,1
149871665,53,51,5,252722,135829,5,0.953694,635393474356,100640489156,252748,135854,5119,619937318982,100625916808,2
149871666,24,26,5,372074,155840,5,0.975285,621949005973,101314120171,372086,155853,5119,621949005974,101299563325,2
149871667,24,29,3,180541,108238,3,0.971786,617150383625,99691847846,180553,108252,5119,617180305548,99706404478,1


## **Fundamental Arkouda Operations**
Most data scientific workflows include preprocessing or main analytical steps that involve sorting data, extracting unique values, generating indices of data that match inequalities, and performing slices on data. We summarize these fundamentals below and then show examples of each:
* **Grouping Operations** -- use `ak.GroupBy()` (`dataframe.groupby()`) to return a `GroupBy` object with information of the array or dataframe such as unique keys, sorted permutation of the indices of the unique keys, and starting offsets (segments) of the unique groups.
* **Vector and Scalar Arithmetic** -- say `A` and `B` are `pdarrays`, supports operations such as `A + A`, `A * 2`, `A == B`, etc.
* **Element-Wise Functions** -- apply some function to every element of an array: absolutes, logs, sin, cos, etc. 
* **Scans and Reductions** -- scans apply a cumulative reduction (`ak.cumsum()`) to every element of a pdarray and reductions return a scalar value (`ak.var()`) to represent an array.
* **(Logical) Indexing** -- given a slice of indices, `pdarray` of type `bool`,  or `pdarray` of type `int`, return a new `pdarray` with the specified indices. 
* **Data Summarization** -- similar to element-wise functions applied directly as a `pdarray` method and functions such as `ak.histogram()`.
* **Sorting** -- sort array data or `co.argsorts()` for multiple arrays.
* **Array Set Operations** -- unions, intersections, differences, etc. of `pdarrays`.
* **Strings** -- search for patterns in multiple strings in a `pdarray`, modify strings, etc. 

### **Grouping Operations**
Using the columns `type_label`, `src_neuron_id`, and `dst_neuron_id` extract the unique links for each type of label that exist for the `connectome_links` dataframe.

In [5]:
cgb = connectome_links.groupby(["src_neuron_id", "dst_neuron_id", "type_label"])
print(f"Number of unique links for edge types: {cgb.unique_keys[0].size}")

Number of unique links for edge types: 147690815


### **Vector and Scalar Arithmetic**

Using the columns `bounding_box_size_x`, `bounding_box_size_y`, and `bounding_box_size_z` we can get the volumes of the cubic size of the bounding box for each link. 

In [6]:
bounding_box_sizes = connectome_links["bounding_box_size_x"] * connectome_links["bounding_box_size_y"] * connectome_links["bounding_box_size_z"]
print(f"Volume taken up by the bounding boxes: {bounding_box_sizes}")

Volume taken up by the bounding boxes: [131936 143220 128304 ... 3120 2088 910]


### **Scans and Reductions**

Extract the maximum size of a bounding box and the link that belongs to it. 

In [7]:
largest_box = ak.argmax(bounding_box_sizes)
print(f"Largest bounding box is owned by link {connectome_links['src_neuron_id'][largest_box]} --> {connectome_links['dst_neuron_id'][largest_box]} with size {bounding_box_sizes[largest_box]}")

Largest bounding box is owned by link 69489386712 --> 60667711330 with size 5820248


### **(Logical) Indexing**

We can index `pdarrays` based off integer or Boolean values. For Boolean indexing, the array used for indexing needs to be the same size and shape as the outer array. A new array is returned with the indexed values. In this example, we find the confidence values greater than 95%.

In [8]:
confidence_to_find = 0.95
boolean_indexing = connectome_links["confidence"] > confidence_to_find
confidences = connectome_links["confidence"]
print(f"There are {len(confidences[boolean_indexing])} links with confidence greater than {confidence_to_find}")

There are 111178305 links with confidence greater than 0.95


### **Value Counts**

We can get the unique value counts of the `type_label` column to see the distribution of the link labels. From the data, there should be 2 labels. Let's see what vallue counts tells us. 

In [9]:
ak.value_counts(connectome_links["type_label"])

(array([1 2]), array([40656728 109214941]))

# **Processing Arkouda Dataframe Data as Graphs**

### **Building an Arachne Graph**

Arachne has the ability to build three different types of graphs `Graph`, `DiGraph`, and `PropGraph` each providing different representations of the data. `Graph` and `DiGraph` can be weighted or unweighted where self-loops are allowed by default and no multiple edges are allowed. A `PropGraph` is a special case of a `DiGraph` that allows attributes to be defined on vertices and edges.

For now, we focus on a `PropGraph` that is essentially a wrapper to an Arkouda dataframe. Edge lists can be transferred to Arachne and through various Arkouda functions and objects like `GroupBy` sort the edge lists into an array-based data structure like the compressed sparse row (CSR) format that explicitily maintains both source and destination vertices called the double index (DI) data structure.

Below, we take the connectome dataframe we loaded in previously

In [10]:
graph = ar.PropGraph()
graph.load_edge_attributes(connectome_links, source_column="src_neuron_id", destination_column="dst_neuron_id", relationship_columns="type_label")

In [11]:
print(f"Number of edges = {graph.size()}")
print(f"Number of nodes = {len(graph)}")

Number of edges = 147071359
Number of nodes = 142660662


In [12]:
out_degrees = graph.out_degree()
in_degrees = graph.in_degree()

In [13]:
print(f"max out degree = {ak.max(out_degrees)}")
print(f"max in degree = {ak.max(in_degrees)}")

max out degree = 442
max in degree = 24301


In [14]:
nodes = graph.nodes()
edges = graph.edges()

In [15]:
nodes

array([243690691 243778373 243778396 ... 105775440656 105775440674 105787807706])

In [16]:
edges

(array([243822242 243836714 243836718 ... 105775440615 105775440615 105787807706]),
 array([258381136 243836718 258394114 ... 105775440656 105775440674 105758678885]))

In [17]:
sorted_out_degrees = ak.argsort(out_degrees)
sorted_in_degrees = ak.argsort(in_degrees)

In [18]:
nodes_sorted_by_out_degree = nodes[sorted_out_degrees]

In [19]:
nodes_sorted_by_in_degree = nodes[sorted_in_degrees]

In [20]:
out_degrees_greater_than_5 = out_degrees > 5
in_degrees_greater_than_5 = in_degrees > 5
nodes_with_out_degrees_greater_than_5 = nodes[out_degrees_greater_than_5]
nodes_with_in_degrees_greater_than_5 = nodes[in_degrees_greater_than_5]

In [21]:
type_2_edges = graph.edge_attributes["type_label"] == 2
bool_edges = type_2_edges & ak.in1d(graph.edge_attributes["src_neuron_id"], nodes_with_out_degrees_greater_than_5) & ak.in1d(graph.edge_attributes["dst_neuron_id"], nodes_with_in_degrees_greater_than_5)
subgraph = graph.edge_attributes[bool_edges]

In [22]:
di_graph = ar.Graph()

In [23]:
di_graph.add_edges_from(subgraph["src_neuron_id"], subgraph["dst_neuron_id"])

In [32]:
len(di_graph)

2728152

In [34]:
di_graph.size()

8713128

In [24]:
di_graph_nodes = di_graph.nodes()
max_out_degree = ak.argmax(di_graph.degree())
max_vertex = di_graph_nodes[max_out_degree]

In [25]:
bfs = ar.bfs_layers(di_graph, int(max_vertex))

In [26]:
bfs_gb = ak.GroupBy(bfs)

In [28]:
unique_keys, count = bfs_gb.count()

In [29]:
unique_keys

array([-1 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46])

In [30]:
count

array([44969 1 3180 59316 464097 667603 629743 402233 217563 127698 66228 21488 7902 3658 2144 1505 1236 961 952 730 792 559 711 460 653 324 352 191 239 146 120 88 66 67 41 46 28 19 10 10 3 6 1 2 1 3 2 5])

In [31]:
ar.triangles(di_graph)

158877

In [36]:
centralities = ar.triangle_centrality(di_graph)

In [37]:
ak.max(centralities)

0.043983710669259865

In [38]:
compat_graph = ar.Graph()
compat_graph.add_edges_from_compat(subgraph["src_neuron_id"], subgraph["dst_neuron_id"])

/scratch/users/oaa9/arkouda-2023.11.15-1.32/arkouda/alignment.py:159: UserWarning: Duplicate terms present in search space. Only first instance of each query term            will be reported.
  warn(


In [57]:
# Probably bad example for k_truss! When edges that do not belong to any triangles are pruned, then the remaining edges must ALL belong to at least 
edges_k_truss = ar.k_truss(compat_graph, 4)

In [58]:
k_truss_gb = ak.GroupBy(edges_k_truss)

In [59]:
unique_keys, counts = k_truss_gb.count()
print(unique_keys)
print(counts)

[-1 3]
[903 8712354]


In [60]:
conn_comps = ar.connected_components(compat_graph)

In [61]:
cc_gb = ak.GroupBy(conn_comps)

In [62]:
unique_keys, counts = cc_gb.count()

In [63]:
print(unique_keys)
print(counts)

[0 40 60 ... 2728073 2728088 2728096]
[2683183 2 15 ... 2 2 4]
